# Deep Learning in Medicine

## BMSC-GA 4493, BMIN-GA 3007

## Lab 9: Autoencoders
credits: Juiting Hsu

## Goal:
- denoising auto-encoder

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import time
import copy

torch.manual_seed(1111);

In [ ]:
if torch.cuda.is_available:
    device = 'cuda'
else:
    device = 'cpu'

## Data

In [ ]:
batch_size = 64
# Data Loader
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                   ])),
    batch_size=batch_size, shuffle=True)

In [ ]:
def plot_digits(digits):
    grids = make_grid(digits, nrow=8)
    plt.figure(figsize=(12, 9))
    plt.imshow(grids.numpy()[0], cmap='gray')

In [ ]:
data, label = next(iter(train_loader))
plot_digits(data)

In [ ]:
plot_digits(data + torch.FloatTensor(data.size()).normal_(mean=0, std=0.6))

## Model: Denoising Auto-encoder

As covered in class, there are mainly two components to an autoencoder: an encoder and a decoder. For this tutorial, we will be using a very simple model with just one hidden layer (not including the output layer), as shown by the image below. The decoder will just be the mirror of the encoder. Rather than learning an undercomplete representation, we will attempt to learn an overcomplete representation by training a [**sparse autoencoder**](https://web.stanford.edu/class/cs294a/sparseAutoencoder_2011new.pdf). In other words, the "code" that our autoencoder learn will be of the same dimension as the input data (28 * 28 = 784). We will, however, add a dropout layer to enforce the sparsity.

Another thing to note is that rather than treating out input as 2-dimensional (28 x 28), we will transform our input to 1-dimensional vectors (784) for simplicity.

<img src="https://cdn-images-1.medium.com/max/1600/1*ZEvDcg1LP7xvrTSHt0B5-Q@2x.png" alt="alt text" width="300" height="120">

Please fill in the code skeleton below to implement your own autoencoder. Think about the dimensions to the components of the modules (encoder and decoder). If you're looking for a more complex architecture, it would probably be better to write separate classes of Encoder and Decoder, then include them both in the model. However, since we're using pretty straightforward MLPs, we can do that in a single class by utilizing the ```nn.Sequential``` module.

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, code_dim, dropout_rate=0.15):
        """
        :param input_dim: dimension of input to autoencoder
        :param hidden_dim: dimension of hidden layer in encoder and decoder
        :param code_dim: dimension of code in autoencoder
        """
        super(AutoEncoder, self).__init__()
        # TODO: initialize your Encoder and Decoder class with the given parameters
        
    def forward(self, x):
        """
        Forward function for the autoencoder. Remember to add dropout to enforce sparsity!
        Return both the output from the decoder and the code from the encoder
        :return output, code
        """
        # TODO: autoencoder forward function

## Training

Next, we will train our autoencoder to denoise handwritten digit image data. Use the functions below and complete the code where necessary. Think about what kind of loss to use as the reconstruction loss for this task.

In [ ]:
def test_model(loader, model, limit=None):
    """
    Help function that tests the models's performance on a dataset
    :param: loader: data loader for the dataset to test against
    """
    loss = 0
    num_samples = 0
    
    model.eval()
    
    predictions = []
    truths = []
    
    for i, (data, labels) in enumerate(loader):
        if i and i==limit:
            break
        data_noise = data + torch.FloatTensor(data.size()).normal_(std=NOISE_STD)
        outputs, _ = model(data_noise.squeeze().view(-1, 28 * 28).to(device))
        # compute loss
        loss += loss_function(outputs, data.view(-1,28*28).to(device)).item() * data.size()[0]
        num_samples += data.size()[0]

    return loss/num_samples

In [ ]:
def train(num_epoch, model):
    for epoch in range(num_epoch):
        for i, (data, labels) in enumerate(train_loader):
            model.train()
            # TO DO: add gaussian noise to data
            
            outputs, _ = model(data_noise.squeeze().view(-1, 28 * 28).to(device))
            model.zero_grad()
            loss = loss_function(outputs, data.view(-1,28*28).to(device))
            loss.backward()
            optimizer.step()

             # report performance
            if (i + 1) % LOG_INTERVAL == 0:
                test_loss = test_model(test_loader, model)
                print('Epoch: [{0}/{1}], Step: [{2}/{3}], Train Loss: {4}, Validation Loss:{5}'.format(
                    epoch + 1, EPOCHS, i + 1, len(train_loader), loss.item(), test_loss))

In [ ]:
EPOCHS = 6
LOG_INTERVAL = 300

LR = 0.002
NOISE_STD = 0.6

model = AutoEncoder(28 * 28, 200, 784).to(device)

# TODO: loss function for reconstruction loss
loss_function = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

train(EPOCHS, model)

## Visualize Results

Let's take a look at how we can use our trained model:

In [ ]:
data, label = next(iter(test_loader))
plot_digits(data)
plt.title("Sampled original data")

In [ ]:
data_noise = data + torch.FloatTensor(data.size()).normal_(mean=0, std=NOISE_STD)
plot_digits(data_noise)
plt.title("Data with Gaussian noise")

In [ ]:
# denoise the data
model.eval()
reconstruction, _ = model(data_noise.view(-1, 28 * 28).to(device))

reconstruction = reconstruction.view(-1, 1, 28, 28).data.cpu()
plot_digits(reconstruction)
plt.title("Reconstructed (denoised) images")

# Visualization of High dimensional vectors using t-SNE

Useful Link to understand t-SNE and how to interpret the visualization: https://distill.pub/2016/misread-tsne/

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(perplexity=50)

In [ ]:
data_np = []
y_np = []
encoded_data_np = []
for data in train_loader:
    encoded_data = model.encoder(data[0].view(-1,28*28).to(device)).cpu().detach().numpy()
    encoded_data_np.append(encoded_data)
    data_np.append(data[0].view(-1,28*28).numpy())
    y_np.append(data[1].numpy())
data_np = np.concatenate(data_np, axis = 0)[:2500,:]
y_np = np.concatenate(y_np)[:2500]
encoded_data_np = np.concatenate(encoded_data_np,axis = 0)[:2500,:]

In [ ]:
red_data = tsne.fit_transform(data_np)

In [ ]:
red_encoded = tsne.fit_transform(encoded_data_np)

In [ ]:
plt.figure(figsize = (10,10))
for i in range(10):
    plt.scatter(red_data[:,0][y_np == i],red_data[:,1][y_np == i], label = str(i), alpha = 0.5)
plt.legend()

In [ ]:
plt.figure(figsize = (10,10))
for i in range(10):
    plt.scatter(red_encoded[:,0][y_np == i],red_encoded[:,1][y_np == i], label = str(i), alpha = 0.5)
plt.legend()